<a href="https://colab.research.google.com/github/Ivelina-Z/Plant-disease-detection/blob/main/plant_disease_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import tensorflow as tf

In [18]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [19]:
import os
import numpy as np
import matplotlib.pyplot as plt
import datetime
import random

## Custom functions

In [20]:
def get_folder_paths(directory):
  folder_names = sorted(os.listdir(directory))
  folder_paths = sorted([os.path.join(directory, name) for name in folder_names])
  return folder_names, folder_paths

## Set TensorBoard

In [21]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [22]:
logdir = "logs/scalars/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir + "/metrics")
file_writer.set_as_default()

In [23]:
tf.random.set_seed(13)
random.seed(13)

The task of that project is to create a model, capable of detecting certain diseases of a tomato plants. The data provided consists of labeled images, including the classes below:

*   Bacterial_spot
*   Early_blight
*   Late_blight
*   Leaf_Mold
*   Septoria_leaf_spot
*   Spider_mites Two-spotted_spider_mite
*   Target_Spot
*   Tomato_Yellow_Leaf_Curl_Virus
*   Tomato_mosaic_virus
*   healthy

To achieve the task a transferable learning with already trained model (ResNet 50) will be used.

<h1 align=center> Tomato plants disease detection </h1>

## Read files and create dataset

The data is divided into two folders. The files in `image` folder are used for train, validation and evaluation of the model, after splitting the data. It consists of 4535 images, which are relatively equally distributed throught the classes. In `manual_testing` folder 5 images of each class are set aside for manual testing with augmentation of the images at the evaluation step.

The structure of each folder is as follows:

*   Image
    *   Tomato___Bacterial_spot
      *   Img_1
      *   Img_2
      *   Img_N
    *   Tomato___Early_blight
      *   Img_1
      *   Img_2
      *   Img_N
    *   Tomato___Late_blight
    *   Tomato___Leaf_Mold
    *   Tomato___Septoria_leaf_spot
    *   Tomato___Spider_mites Two-spotted_spider_mite
    *   Tomato___Target_Spot
    *   Tomato___Tomato_Yellow_Leaf_Curl_Virus
    *   Tomato___Tomato_mosaic_virus
    *   Tomato___healthy

*   Manual_testing


In [24]:
MAIN_DIR = '/content/gdrive/MyDrive/datasets/plant_disease_dataset/image'
MANUAL_TEST_DIR = '/content/gdrive/MyDrive/datasets/plant_disease_dataset/manual_testing'

In [25]:
folder_names, folder_paths = get_folder_paths(MAIN_DIR)
folder_paths

['/content/gdrive/MyDrive/datasets/plant_disease_dataset/image/Tomato___Bacterial_spot',
 '/content/gdrive/MyDrive/datasets/plant_disease_dataset/image/Tomato___Early_blight',
 '/content/gdrive/MyDrive/datasets/plant_disease_dataset/image/Tomato___Late_blight',
 '/content/gdrive/MyDrive/datasets/plant_disease_dataset/image/Tomato___Leaf_Mold',
 '/content/gdrive/MyDrive/datasets/plant_disease_dataset/image/Tomato___Septoria_leaf_spot',
 '/content/gdrive/MyDrive/datasets/plant_disease_dataset/image/Tomato___Spider_mites Two-spotted_spider_mite',
 '/content/gdrive/MyDrive/datasets/plant_disease_dataset/image/Tomato___Target_Spot',
 '/content/gdrive/MyDrive/datasets/plant_disease_dataset/image/Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 '/content/gdrive/MyDrive/datasets/plant_disease_dataset/image/Tomato___Tomato_mosaic_virus',
 '/content/gdrive/MyDrive/datasets/plant_disease_dataset/image/Tomato___healthy']

In [26]:
class_names = {label: name.split('___')[-1] for label, name in enumerate(folder_names)}
class_names

{0: 'Bacterial_spot',
 1: 'Early_blight',
 2: 'Late_blight',
 3: 'Leaf_Mold',
 4: 'Septoria_leaf_spot',
 5: 'Spider_mites Two-spotted_spider_mite',
 6: 'Target_Spot',
 7: 'Tomato_Yellow_Leaf_Curl_Virus',
 8: 'Tomato_mosaic_virus',
 9: 'healthy'}

In [27]:
sum([len(os.listdir(p)) for p in folder_paths])

4535

In [28]:
{p: len(os.listdir(p)) for p in folder_paths}

{'/content/gdrive/MyDrive/datasets/plant_disease_dataset/image/Tomato___Bacterial_spot': 420,
 '/content/gdrive/MyDrive/datasets/plant_disease_dataset/image/Tomato___Early_blight': 475,
 '/content/gdrive/MyDrive/datasets/plant_disease_dataset/image/Tomato___Late_blight': 458,
 '/content/gdrive/MyDrive/datasets/plant_disease_dataset/image/Tomato___Leaf_Mold': 465,
 '/content/gdrive/MyDrive/datasets/plant_disease_dataset/image/Tomato___Septoria_leaf_spot': 431,
 '/content/gdrive/MyDrive/datasets/plant_disease_dataset/image/Tomato___Spider_mites Two-spotted_spider_mite': 430,
 '/content/gdrive/MyDrive/datasets/plant_disease_dataset/image/Tomato___Target_Spot': 452,
 '/content/gdrive/MyDrive/datasets/plant_disease_dataset/image/Tomato___Tomato_Yellow_Leaf_Curl_Virus': 485,
 '/content/gdrive/MyDrive/datasets/plant_disease_dataset/image/Tomato___Tomato_mosaic_virus': 443,
 '/content/gdrive/MyDrive/datasets/plant_disease_dataset/image/Tomato___healthy': 476}